In [3]:
import pandas as pd

imdb_path = "data/imdb/imdb.csv"
data = pd.read_csv(imdb_path)

In [4]:
data

,review,sentiment
0,One of the other reviewers has mentioned that ...,positive
1,A wonderful little production. <br /><br />The...,positive
2,I thought this was a wonderful way to spend ti...,positive
3,Basically there's a family where a little boy ...,negative
4,"Petter Mattei's ""Love in the Time of Money"" is...",positive
5,"Probably my all-time favorite movie, a story o...",positive
6,I sure would like to see a resurrection of a u...,positive
7,"This show was an amazing, fresh & innovative i...",negative
8,Encouraged by the positive comments about this...,negative
9,If you like original gut wrenching laughter yo...,positive


In [5]:
train_x = list(data["review"])
train_y = list(data["sentiment"])

In [10]:
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
import numpy as np

max_len = 100
max_words = 10000

tokenizer = Tokenizer(num_words=max_words)
tokenizer.fit_on_texts(train_x)
sequences = tokenizer.texts_to_sequences(train_x)

word_index = tokenizer.word_index
print("고유한 토큰 수 : {}".format(len(word_index)))

고유한 토큰 수 : 124252


In [15]:
train_x = pad_sequences(sequences, maxlen=max_len)
train_y = np.array(train_y)

In [28]:
train_x, test_x = train_x[:40000], train_x[40000:]
train_y, test_y = train_y[:40000], train_y[40000:]

In [38]:
glove_path = "data/glove.6B/glove.6B.100d.txt"

embedding_index = {}
f = open(glove_path, "r", encoding="utf-8")
for line in f:
    values = line.split()
    word = values[0]
    vector = np.array(values[1:], dtype="float32")
    embedding_index[word] = vector

In [69]:
embedding_dim = 100

embedding_matrix = np.zeros(shape=(max_words, embedding_dim))
for word, i in word_index.items():
    if i < max_words:
        embedding_vector = embedding_index.get(word)
        if embedding_vector is not None:
            embedding_matrix[i] = embedding_vector

In [73]:
from keras.models import Sequential
from keras.layers import Dense, Embedding, Flatten

model = Sequential()
model.add(Embedding(input_dim=max_words, output_dim=embedding_dim, input_length=max_len))
model.add(Flatten())
model.add(Dense(32, activation="relu"))
model.add(Dense(1, activation="sigmoid"))
model.summary()

W0816 21:02:02.750766  8960 deprecation_wrapper.py:119] From C:\Users\jeon\AppData\Local\Programs\Python\Python36-32\Lib\conda\lib\site-packages\keras\backend\tensorflow_backend.py:74: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.get_default_graph instead.

W0816 21:02:02.986184  8960 deprecation_wrapper.py:119] From C:\Users\jeon\AppData\Local\Programs\Python\Python36-32\Lib\conda\lib\site-packages\keras\backend\tensorflow_backend.py:517: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.

W0816 21:02:03.053704  8960 deprecation_wrapper.py:119] From C:\Users\jeon\AppData\Local\Programs\Python\Python36-32\Lib\conda\lib\site-packages\keras\backend\tensorflow_backend.py:4138: The name tf.random_uniform is deprecated. Please use tf.random.uniform instead.



_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 100, 100)          1000000   
_________________________________________________________________
flatten_1 (Flatten)          (None, 10000)             0         
_________________________________________________________________
dense_1 (Dense)              (None, 32)                320032    
_________________________________________________________________
dense_2 (Dense)              (None, 1)                 33        
Total params: 1,320,065
Trainable params: 1,320,065
Non-trainable params: 0
_________________________________________________________________


In [79]:
model.layers[0].set_weights([embedding_matrix])
model.layers[0].trainable = False

W0816 21:03:46.377378  8960 deprecation_wrapper.py:119] From C:\Users\jeon\AppData\Local\Programs\Python\Python36-32\Lib\conda\lib\site-packages\keras\backend\tensorflow_backend.py:174: The name tf.get_default_session is deprecated. Please use tf.compat.v1.get_default_session instead.

W0816 21:03:46.383379  8960 deprecation_wrapper.py:119] From C:\Users\jeon\AppData\Local\Programs\Python\Python36-32\Lib\conda\lib\site-packages\keras\backend\tensorflow_backend.py:181: The name tf.ConfigProto is deprecated. Please use tf.compat.v1.ConfigProto instead.



In [81]:
model.compile(optimizer="Adam", loss="binary_crossentropy", metrics=["acc"])
history = model.fit(x=train_x, y=train_y, epochs=10, batch_size=12, validation_data=(test_x, test_y))

Train on 40000 samples, validate on 10000 samples
Epoch 1/10
40000/40000 [==============================] - 39s 966us/step - loss: 0.5602 - acc: 0.7085 - val_loss: 0.5510 - val_acc: 0.7185
Epoch 2/10
40000/40000 [==============================] - 37s 916us/step - loss: 0.4819 - acc: 0.7683 - val_loss: 0.5204 - val_acc: 0.7425
Epoch 3/10
40000/40000 [==============================] - 44s 1ms/step - loss: 0.4502 - acc: 0.7898 - val_loss: 0.5204 - val_acc: 0.7413
Epoch 4/10
40000/40000 [==============================] - 37s 931us/step - loss: 0.4208 - acc: 0.8059 - val_loss: 0.5306 - val_acc: 0.7450
Epoch 5/10
40000/40000 [==============================] - 37s 914us/step - loss: 0.3943 - acc: 0.8178 - val_loss: 0.5638 - val_acc: 0.7356
Epoch 6/10
40000/40000 [==============================] - 37s 913us/step - loss: 0.3717 - acc: 0.8277 - val_loss: 0.5837 - val_acc: 0.7347
Epoch 7/10
40000/40000 [==============================] - 37s 926us/step - loss: 0.3523 - acc: 0.8338 - val_loss: 0.63

In [82]:
model.save_weights("model/keras_imdb.h5")